## 1. 計算所有車站*各村里距離

In [21]:
import pandas as pd
import numpy as np
from geopy import distance

### 讀取所有車站點位


In [22]:
# 讀取車站資料
df_sta = pd.read_csv('../data/all_stations_finished.csv', encoding='utf-8')
# mrt['Coordinate'] = [i for i in zip(mrt['lat'],mrt['lng'])]
print(df_sta.groupby('車站狀態').size(),'\n')

index = df_sta[df_sta.duplicated('車站名稱', keep=False)].index.copy()
print('重複的站點數量:',df_sta.loc[index,'車站名稱'].nunique())
print('重複的站名:',df_sta.loc[index,'車站名稱'].unique())

車站狀態
0    119
1    217
dtype: int64 

重複的站點數量: 33
重複的站名: ['台北捷運頂埔站' '台北捷運土城站' '台北捷運板橋站' '台北捷運西門站' '台北捷運台北車站' '台北捷運忠孝新生站'
 '台北捷運忠孝復興站' '台北捷運南港展覽館站' '台北捷運動物園站' '台北捷運大安站' '台北捷運南京復興站' '台北捷運大坪林站'
 '台北捷運古亭站' '台北捷運中正紀念堂站' '台北捷運中山站' '台北捷運松江南京站' '台北捷運中和站' '台北捷運迴龍站'
 '台北捷運景安站' '台北捷運東門站' '台北捷運民權西路站' '台北捷運頭前庄站' '台北捷運紅樹林站' '桃園捷運坑口站' '桃園捷運橫山站'
 '桃園捷運環北站' '桃園捷運大興西路站' '台鐵桃園站' '台鐵內壢站' '台鐵中壢站' '台鐵埔心站' '台鐵楊梅站' '台鐵富岡站']


##### 車站資料探索

In [7]:
# df_sta.loc[index][df_sta['車站種類']=='train']
# df_sta[df_sta['車站名稱']=='台北捷運土城站']
# df_sta[df_sta['車站名稱']=='桃園捷運大興西路站']
# df_s[df_s['車站名稱']=='台北捷運台北車站']

### 讀取鄰里資料

In [23]:
# 讀取鄰里資料
vill = pd.read_csv('../data/pop_merge.csv', index_col ='Village_ID' ,  encoding='utf-8')
# vill['Coordinate'] = [ i for i in zip(vill['lat'],vill['lng'])]
vill.dropna(inplace=True)
vill.head(3)

,Country,District,Village,Village_Chief,Address,lat,lng,納稅單位,綜合所得總額,所得平均,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
Village_ID,,,,,,,,,,,,,,,,,,,,,
10017010-001,基隆市,中正區,德義里,張建源,基隆市中正區義二路97之 1號,25.132788,121.746435,275.0,23622.5,85.9,...,240,70,41,6,297,347,2.0,13.0,37,35
10017010-002,基隆市,中正區,正義里,駱錫賢,基隆市中正區義二路147號,25.133882,121.747188,393.0,27012.9,68.7,...,396,166,128,13,516,588,3.0,19.0,65,57
10017010-003,基隆市,中正區,信義里,江添丁,基隆市中正區正義路69號,25.134521,121.748392,257.0,19097.9,74.3,...,310,103,100,14,391,405,2.0,11.0,26,41


### 運算流程
1. 計算所有捷運站*里辦公室的距離(n= 2149)  <br>
2. 將每個捷運站存為一個list / numpy array /series <br>
3. 如果距離小於3公里, 則以{里名：距離} 格式 ,存入每個捷運站的list內 <br>
4. 將捷運站名與 各里距離  以{捷運站名：各里距離} 存為json檔案 <br>  


In [ ]:
station_vil_dis = {}
for s in range(len(df_sta['車站編號'])):  # 從1開始 ，台北車站略過未抓
    #     if s>2:break

    vil_dis ={} # 建立空字典 
    for v in range(1,len(vill['Village'])):
        sta_name = (df_sta['車站種類'][s] + ',' + 
                    df_sta['車站編號'][s] + ',' + 
                    df_sta['車站名稱'][s])
        vill_name = (vill['Country'].iloc[v] + 
                     vill['District'].iloc[v] + 
                     vill['Village'].iloc[v])
        
        # 計算 '捷運站' 與 '村里辦公室'的距離
        mrt_cor  = (df_sta['lat'][s],df_sta['lng'][s])
        vill_cor = (vill['lat'].iloc[v],vill['lng'].iloc[v])
        dis = distance.distance( mrt_cor , vill_cor).km 
        
#         print(v+1,f'{sta_name}-{vill_name},distnce:{dis:.3f}',end='\r')
                    
        if dis<=3:  # 當距離小於n的時候才寫入
            vil_dis.update({vill_name : round(dis,3)})
#             print(f'{sta_name}-{vill_name},{dis:.3f}',end='\r')
     
    station_vil_dis.update({sta_name : vil_dis })
    
    # check progess
    progress  = (s+1)/len(df_sta)*100
  
    print(f'progress ：{s+1} , {progress:.2f}％',end='\r') # print out 完成 %數
#     print(f'{sta_name}-{len(vil_dis)}')
    
output = pd.Series(station_vil_dis)
output.to_json('../data/mrt_village_dis.json')

## 二 計算各車站人口結構權重

### 計算權重流程
1. 讀入車站資料，取得所有車站名單 <br>
2. 讀入計算好距離的各車站與各里距離的json檔 <br>
3. 以迴圈方式一一指定各站點名稱 <br>
4. 以站點名稱作為 key,取得各車站與里之間的距離，使用該距離值去計算權重


In [24]:
# 讀入json檔
mrt_village_dis = pd.read_json('../data/mrt_village_dis.json', typ='series',encoding='utf-8')
mrt_village_dis.head(3)

tp_mrt,BL01,台北捷運頂埔站    {'新北市板橋區深丘里': 1.6, '新北市板橋區溪洲里': 2.995, '新北市板橋區...
tp_mrt,BL02,台北捷運永寧站    {'新北市板橋區深丘里': 0.527, '新北市板橋區溪洲里': 2.471, '新北市板...
tp_mrt,BL03,台北捷運土城站    {'新北市板橋區深丘里': 1.279, '新北市板橋區和平里': 2.859, '新北市板...
dtype: object

In [25]:
# 讀取彙整後的人口結構資料
pop = pd.read_csv('../data/pop_merge.csv',   encoding='utf-8')

# 將行政區里設為index 方便之後迴圈抓權重使用
pop['vill_name'] = pop['Country'] + pop['District'] + pop['Village']
pop.set_index('vill_name',inplace=True)
pop.head(3)

,Country,District,Village,Village_ID,Village_Chief,Address,lat,lng,納稅單位,綜合所得總額,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
vill_name,,,,,,,,,,,,,,,,,,,,,
基隆市中正區德義里,基隆市,中正區,德義里,10017010-001,張建源,基隆市中正區義二路97之 1號,25.132788,121.746435,275.0,23622.5,...,240,70,41,6,297,347,2.0,13.0,37,35
基隆市中正區正義里,基隆市,中正區,正義里,10017010-002,駱錫賢,基隆市中正區義二路147號,25.133882,121.747188,393.0,27012.9,...,396,166,128,13,516,588,3.0,19.0,65,57
基隆市中正區信義里,基隆市,中正區,信義里,10017010-003,江添丁,基隆市中正區正義路69號,25.134521,121.748392,257.0,19097.9,...,310,103,100,14,391,405,2.0,11.0,26,41


In [ ]:
# 先以npArray/Series格式存取資料逐條累加成為多維陣列，最後再進行堆疊/轉置
# 將所有取得的資料存成list 最後再以pd.concat(data=list, axis=1) 建立Dataframe資料
# 資料讀寫均採用 npArray格式以加快運算

a1 = (df.iloc[0,7:] + df.iloc[1,7:]).values.reshape(-1,34)
a2 = (df.iloc[2,7:] + df.iloc[3,7:]).values.reshape(-1,34)
a3 = (df.iloc[4,7:] + df.iloc[5,7:]).values.reshape(-1,34)
print(a1.shape)

dataset = [a1,a2,a3] # 先建立空串列，再遞迴把資料加入 
a_new = np.concatenate(dataset, axis=0)
df_new = pd.DataFrame(data = a_new ,
                      index = mrt_village_dis.index[:3],
                      columns = [df.columns[7:]])
df_new.index.names = ['Station_name']
print(a_new.shape)

dataset

In [26]:
columns  = pop.columns[8:]   # 取出人口結構欄位名稱
dis_t    = np.exp(-dis)      # e^(-dis) 將距離取自然對數轉換
pop_list = []                # 裝pop_weighted
n_vill_list = []             # 裝鄰里辦公室數量


for i in range(len(mrt_village_dis)):
    
    stat_name = mrt_village_dis.index[i]   # 取所有車站名稱
    vill_list = mrt_village_dis[i]         # 取所有區里名稱       
    n_vill = len(vill_list)                # 取所有區里數量(距離車站n公里內)
    
    # 建立維度(1,35) 值為0的陣列準備承接權重計算結果
    pop_weighted = np.zeros((1,len(columns)))        # nparray 欄位數量取自人口結構

#     if i == 3: break    
#     print(f'{i+1:2d}, 里辦公室數:{n_vill:3d},{stat_name}')

    for j in range( n_vill ):
        vill_name = list(vill_list.keys())[j]    # 村里名  
        dis       = list(vill_list.values())[j]  # 捷運站與里辦公室距離
        
        
#         if j == 3: break        
#         print(f'\t{j+1},{vill_name} : {dis:.3f}')
        
        try:
            # 根據里辦公室與捷運站距離做權重轉換
            pop_weighted += ( pop.loc[vill_name,'納稅單位':]
                             .apply(lambda x: round(x*dis_t,0)) )  
             
        except Exception as e:
            print(f'{j}-{vill_name} : {ｅ}')
        
    try:
        pop_list.append(pop_weighted)
        n_vill_list.append(n_vill)
        
    except Exception as e:
        print(f'{i}-{stat_name}\t: {ｅ}')
    
    # check progess
    progress  = (i+1)/len(mrt_village_dis)*100
    print(f'progress ：{i+1}-{progress:.2f}％.',end='\r') # print out 完成 %數


# 將各車站轉換後的人口結構資料存為npArray格式
stat_pop = np.concatenate(pop_list, axis=0) 

# 將各車站轉換後的人口結構資料存為Dataframe格式
df_stat_pop = pd.DataFrame(data = stat_pop ,    
#                       index = mrt_village_dis.index,
                      columns = columns)


stat_list = [x.split(',') for x in list(mrt_village_dis.index)] # mrt_village_dis.json 取出各車站名稱

df = pd.concat(
    [pd.DataFrame(stat_list, columns =['車站種類','車站編號','車站名稱'] ),  # 加入車站屬性集車站名
     pd.Series(n_vill_list, name='村里數量'),                               
     df_stat_pop ], 
    axis=1) 

df.to_csv('../data/station_pop_weighted.csv',encoding='utf-8')

df

,車站種類,車站編號,車站名稱,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
0,tp_mrt,BL01,台北捷運頂埔站,30,15072.0,1270678.0,852.0,19790.0,53554.0,26650.0,...,14992.0,6660.0,4264.0,255.0,23934.0,22741.0,412.0,322.0,2038.0,2104.0
1,tp_mrt,BL02,台北捷運永寧站,49,25130.0,2148284.0,1444.0,32794.0,88407.0,43516.0,...,24014.0,10546.0,6715.0,394.0,39509.0,37788.0,692.0,482.0,3394.0,3407.0
2,tp_mrt,BL03,台北捷運土城站,63,33535.0,2925024.0,1888.0,43504.0,117022.0,57370.0,...,31339.0,13788.0,8767.0,503.0,52125.0,50177.0,903.0,614.0,4633.0,4468.0
3,tp_mrt,BL04,台北捷運海山站,105,50931.0,4495021.0,3192.0,66921.0,177725.0,87046.0,...,46627.0,20473.0,14098.0,790.0,78269.0,76753.0,1357.0,986.0,7020.0,6696.0
4,tp_mrt,BL05,台北捷運亞東醫院站,141,64662.0,5784559.0,4359.0,86239.0,227308.0,111046.0,...,58136.0,25392.0,18141.0,1074.0,100014.0,98450.0,1696.0,1294.0,8817.0,8554.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,train,R7,台鐵中壢站,67,32243.0,3101584.0,2231.0,43000.0,114478.0,56162.0,...,29966.0,8738.0,6972.0,472.0,50270.0,49685.0,1135.0,638.0,5022.0,4694.0
332,train,R8,台鐵平鎮站(未確定站址),57,26536.0,2567001.0,1897.0,34696.0,94117.0,46436.0,...,24675.0,7284.0,5811.0,364.0,41220.0,41275.0,954.0,502.0,4227.0,3998.0
333,train,R9,台鐵埔心站,22,11299.0,1016846.0,681.0,14358.0,39912.0,19817.0,...,11098.0,3287.0,2350.0,140.0,17529.0,17579.0,454.0,169.0,2096.0,1413.0
334,train,R10,台鐵楊梅站,23,9288.0,823292.0,702.0,11564.0,32515.0,16229.0,...,9364.0,2764.0,1972.0,166.0,14232.0,14214.0,344.0,182.0,1562.0,1064.0


In [27]:
df[df['車站名稱'].str.contains(r'台北車?站')]

,車站種類,車站編號,車站名稱,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
11,tp_mrt,BL12,台北捷運台北車站,169,90114.0,11146716.0,6800.0,116593.0,274167.0,130527.0,...,56426.0,18728.0,17044.0,1013.0,117062.0,121308.0,2151.0,2069.0,15641.0,17864.0
137,tp_mrt,R10,台北捷運台北車站,169,90114.0,11146716.0,6800.0,116593.0,274167.0,130527.0,...,56426.0,18728.0,17044.0,1013.0,117062.0,121308.0,2151.0,2069.0,15641.0,17864.0
203,ty_mrt,A1,桃園捷運台北車站,173,85178.0,9943984.0,6507.0,113224.0,265053.0,126935.0,...,57770.0,20664.0,18200.0,1095.0,113542.0,115342.0,2070.0,2036.0,14625.0,16719.0
281,train,1000,台鐵台北站,168,88026.0,10804212.0,6688.0,114138.0,268341.0,127855.0,...,55729.0,18657.0,16960.0,1009.0,114613.0,118474.0,2110.0,2036.0,15228.0,17403.0
322,thsr,Taipei,高鐵台北站,168,88026.0,10804212.0,6688.0,114138.0,268341.0,127855.0,...,55729.0,18657.0,16960.0,1009.0,114613.0,118474.0,2110.0,2036.0,15228.0,17403.0


In [28]:
df[df['車站名稱'].str.contains(r'板橋車?站')]

,車站種類,車站編號,車站名稱,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
6,tp_mrt,BL07,台北捷運板橋站,168,73422.0,6782623.0,5319.0,99337.0,256451.0,125120.0,...,63900.0,27948.0,21413.0,1341.0,112082.0,111689.0,1856.0,1534.0,9910.0,9314.0
186,tp_mrt,Y16,台北捷運板橋站,167,71742.0,6625236.0,5281.0,97386.0,251562.0,122783.0,...,62730.0,27573.0,21403.0,1362.0,109837.0,109452.0,1815.0,1540.0,9439.0,9200.0
286,train,1020,台鐵板橋站,165,70993.0,6557608.0,5218.0,96454.0,248749.0,121356.0,...,61972.0,27196.0,21036.0,1321.0,108637.0,108198.0,1788.0,1510.0,9542.0,9034.0
323,thsr,Banqiao,高鐵板橋站,168,73382.0,6775414.0,5317.0,99343.0,256340.0,125052.0,...,63903.0,27980.0,21483.0,1349.0,112014.0,111625.0,1852.0,1542.0,9894.0,9305.0


In [30]:
df.describe()

,村里數量,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,女性人口,0-14歲人口,15-64歲人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
count,336.000000,336.000000,3.360000e+02,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,...,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.00000,336.000000,336.000000,336.00000
mean,76.375000,39679.738095,4.379261e+06,2770.645833,51188.916667,128411.068452,61829.598214,66583.565476,16909.101190,90647.973214,...,29273.202381,10600.773810,8547.583333,551.074405,55381.967262,56852.898810,1004.31250,828.586310,6081.184524,6332.93750
std,55.462365,28839.974286,3.667914e+06,2170.414490,36525.660498,88729.583958,42227.440898,46535.680934,11439.572893,60934.188644,...,19161.005556,7712.505524,6041.290072,361.334116,37955.743807,39675.991674,647.13333,628.887911,4539.553762,5237.73973
min,3.000000,102.000000,6.789000e+03,77.000000,249.000000,556.000000,311.000000,241.000000,27.000000,358.000000,...,146.000000,129.000000,105.000000,24.000000,169.000000,228.000000,1.00000,9.000000,15.000000,21.00000
25%,28.000000,12895.750000,1.248360e+06,872.250000,18812.750000,47713.500000,23456.250000,24342.750000,6593.250000,34769.000000,...,12591.250000,4551.750000,3527.000000,258.750000,21222.000000,20539.750000,391.75000,288.750000,2494.250000,1997.75000
50%,62.000000,34080.500000,3.397572e+06,2039.000000,43504.500000,111914.500000,54462.000000,57465.000000,15473.500000,81178.000000,...,26761.000000,8649.000000,7192.000000,479.000000,49184.500000,49502.000000,956.00000,627.500000,4836.500000,4674.50000
75%,127.000000,66579.500000,6.559260e+06,4713.750000,86388.500000,213780.750000,103565.000000,110508.000000,25932.500000,149949.250000,...,46218.000000,15469.000000,12759.500000,781.000000,91957.250000,92988.000000,1583.75000,1416.750000,9257.500000,9316.25000
max,201.000000,101870.000000,1.426274e+07,7321.000000,126283.000000,308006.000000,148209.000000,159804.000000,41891.000000,217680.000000,...,73920.000000,31163.000000,24137.000000,1590.000000,130798.000000,137056.000000,2238.00000,2179.000000,17181.000000,19192.00000
